<font size="6">Revised Simos method for calculation of criteria weights</font>

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/cghiaus/ELECTRE_Tri/HEAD?labpath=Simos_revised%2FSimos_revised_explained.ipynb)



This notebook explains the function `criteria_weights_Simos` from the file `Simos_revised.py`. The explanation follows the script `Simos_revised_script.py`.

In [1]:
"""
Append `src/` directory to `path`
"""
import sys
import os

notebook_dir = os.path.dirname(os.path.abspath('__file__'))
project_root = os.path.dirname(os.path.dirname(notebook_dir))

src_dir = os.path.join(project_root, 'src')
sys.path.append(src_dir)

In [2]:
import numpy as np
import pandas as pd

import Simos_revised

# Principles of the method

The method lets the decision maker to hierarchise the criteria by obtaining relative weights that add up to 100. The decision maker is asked:
- to order the criteria in categories from least important to most important (some criteria can be ex aequo) with the possibility to indicate different gaps between successive categories;  
- to give a ratio between the importance of the most and least important category.

## Procedure

1. The decision maker receives:
    - a set of **cards** containing the name of each criterion (with some complementary information, if needed).
    - a set of **white cards**.
2. The decision maker is asked to rank these cards (i.e. criteria) from the least importnant to the most important (in ascending order). If more criteria have the same importance (i.e. same weight), the cards form a subset (they are bound together).
3. The decision maker is asked to introduce white cards between two cards (or subsets of cards) if the difference is considered more impornant:
    - *no white card*: a unit of difference between subsets of criteria;
    - *one white card*: difference of two units between criteria;
    - $n$ *white cards*: difference of $n+1$ units between criteria.
4. The decision maker states how many times the last criterion is more important than the first one in the ranking. This value is denoted by $z$.

## Example 1

Let us consider the example given by Papathanasiou, J., Ploskas, N. (2018). The set of 8 criteria: {a, b, c, d, e, f, g, h} and the ranking obtained by following the procedure are presented in Table A.1, page 167 and the normalized weights (that sum up to 100) for $z = 6.5$ are presented in Table A.2 page 168.

|Position| Subsets |Normalized weights|
|--------|---------|------------------|
| 0      |   b, d  |2.61437908497     |
| 1      |   c     |6.2091503268      |
| 2      |  white  |                  |
| 3      | e, f, h |13.3986928105     |
| 4      |  white  |                  |
| 5      |  white  |                  |
| 6      |  a, g   |24.1830065359     |

Note:

In [3]:
# Ratio between the most important and the least important criteria
24.1830065359 / 2.61437908497

9.24999999997227

In [4]:
# Sum of all weights:
2 * 2.61437908497 + 6.2091503268 + 3 * 13.3986928105 + 2 * 24.1830065359

100.00000000003999

In [5]:
# Sets with white cards included (positions)
sets_white = pd.read_csv("../../data/cards_subsets_1.csv")

#------------display------------
df = sets_white
df = df.fillna("")
df.index.name = 'Position'
print('Information given by the set of cards')
df

Information given by the set of cards


,0,1,2
Position,,,
0,b,d,
1,c,,
2,white,,
3,e,f,h
4,white,,
5,white,,
6,a,g,


## Example 2

Let us consider the example given by Figueira, J., Roy, B. (2002). The set of 12 criteria: {a, b, c, d, e, f, g, h, i, j, k, l} and the ranking obtained by follwing the procedure are presented in Table 1, page 319 and Table 2 page 321. The obtained normalized weights (that sum up to 100) for $z = 6.5$ are presented in Table 5 page 324.

|Position| Subsets   |Normalized weights|
|--------|-----------|------------------|
| 0      | c, g, l   |2.366863905       |
| 1      | d         |4.544378698       | 
| 2      | white     |                  |
| 3      | b, f, i, j|8.875739645       |
| 4      | e         |11.053254438      |
| 5      | a, h      |13.207100592      |
| 6      | k         |15.384615385      |

Note:

In [6]:
# Ratio between the most important and the least important criteria
15.384615385 / 2.366863905

6.500000001056249

In [7]:
# Sum of all weights:
3 * 2.366863905 + 4.544378698 + 4 * 8.875739645 + 11.053254438 + 2 * 13.207100592 + 15.384615385

99.99999999999999

In [8]:
# Figueira, J., Roy, B. (2002) Table 1, page 319 and Table 2 page 321
# Sets with white cards included (positions)
sets_white = pd.read_csv("../../data/cards_subsets_2.csv")

#------------display------------
df = sets_white
df.index.name = 'Position'
df = df.fillna("")
print('Information with position given by the set of cards')
df

Information with position given by the set of cards


,0,1,2,3
Position,,,,
0,c,g,l,
1,d,,,
2,white,,,
3,b,f,i,j
4,e,,,
5,a,h,,
6,k,,,


## Ratio between the most and the least importat criteria 
Suppose that the last subset is 6.5 times more important than the first one, i.e.  $z = 6.5$:

In [9]:
# The last subset is z more important than the 1st one
z = 6.5

# Revised Simos method

## Chose subsets

- `subsets_1` from Papathanasiou, J., Ploskas, N. (2018), Table A.1, page 167 
- `subsets_2` from Figueira, J., Roy, B. (2002), Table 1, page 319:

In [10]:
set_cards_file = "../../data/cards_subsets_1.csv"
sets_white = pd.read_csv(set_cards_file)

## Non-normalized weights

### Unitary ratio between two consecutive ranks

Let $e'_r$ be the number of white cars between the ranks $r$ and $r+1$. Then (Figueira, Roy 2002 eq. §3.2.1 and Papathanasiou, Ploskas, 2018, eq. (A.1)):

$$
\left\{\begin{matrix}
e_r = e'_r+1\\ 
e=\sum_{r-1}^{n-1}e_r\\ 
u=\frac{z-1}{e}
\end{matrix}\right.
$$

In [11]:
# Sets without white cards (ranks)
sets = sets_white[~sets_white["0"].str.contains("white")]

#------------display------------
df = sets.copy()
df.index.name = 'Rank'
df = df.fillna("")
print('Ranks of criteria without white cards')
df

Ranks of criteria without white cards


,0,1,2
Rank,,,
0,b,d,
1,c,,
3,e,f,h
6,a,g,


In [12]:
# number of elements in each subset
c = sets.count(axis=1)

#------------display------------
df['c'] = c
df

,0,1,2,c
Rank,,,,
0,b,d,,2
1,c,,,1
3,e,f,h,3
6,a,g,,2


In [13]:
# unitary ratio between two consecutive ranks
# the range is actually z - 1
u = round((z - 1) / len(c), 6)

#------------display------------
print(f"u = {u:.3f}")

u = 1.375


### Vector of differences between subsets

The 1st difference is $e_0 = 0$. The difference between two positions (including white cards) is *1*. The difference between ranks include the white cards.

In [14]:
# differences e (écarts?) of indexes of non-'white' in subsets
idx_diff = np.diff(sets.index.to_numpy())
e = np.append(0, idx_diff)

#------------display------------
df['e'] = e
df

,0,1,2,c,e
Rank,,,,,
0,b,d,,2,0
1,c,,,1,1
3,e,f,h,3,2
6,a,g,,2,3


### Non-normalized weights $k_r$

$$k_r = 1 + u \sum_{i=0}^{r-1}e_i$$

in which $e_0 = 0$.

The ratio of weigts of last and first criteria is 
$$ \frac{k_{r,n}}{k_{r,0}} \cong z$$

In [15]:
# Non-normalized weights k
k = 1 + u * np.cumsum(e)

#------------display------------
df['k'] = k
df

,0,1,2,c,e,k
Rank,,,,,,
0,b,d,,2,0,1.000
1,c,,,1,1,2.375
3,e,f,h,3,2,5.125
6,a,g,,2,3,9.250


## Normalized weights

The wieghts are normalized to add up to 100:

$$k_{r}^{*} = \frac{100}{\sum_{i=1}^{\widetilde{n}}c_i k_i}k_r$$

In [16]:
# Normalized weights
weights = 100 / sum(c * k) * k

#------------display------------
df['weights'] = weights
df

,0,1,2,c,e,k,weights
Rank,,,,,,,
0,b,d,,2,0,1.000,2.614379
1,c,,,1,1,2.375,6.209150
3,e,f,h,3,2,5.125,13.398693
6,a,g,,2,3,9.250,24.183007


In [17]:
print(f"Sum of normalized weights: {sum(c * df['weights']):.2f}")

Sum of normalized weights: 100.00


## Weights allocated to each criterion
The weights are associated to a set of criteria *ex aequo* (see above). In order to obtain pairs [Criteria, Normed_weight]: 
- flat the criteria,
- expand and flat the weigths,
- make a dataframe which associates each criterion with its corresponding weight.

In [18]:
# Reshape the results to match each criterion with its weight
# flat subsets
sets_flat = sets.stack().values

# weights expanded to correspond to elemnets of sets_flat
weights_flat = [c.values[i] * [weights[i]] for i in range(len(c))]
weights_flat = sum(weights_flat, [])

# return the result as a dataframe
df = pd.DataFrame({'Criteria': sets_flat, 'Weight': weights_flat})

# Result validation
Compare the results obtained above with those presented in:
- Papathanasiou, J., Ploskas, N. (2018), Table A.2 page 168 for `subsets_1`
- Figueira, J., Roy, B. (2002), Table 5, column *Normalized weights $k^*_i$* (note that in this table, the results are in the order of the weights, not of the criteria) for `subsets_2`.

In [19]:
# Total of weights (should be 100).
Total = df['Weight'].sum()
df.loc['Total', 'Weight'] = Total

In [20]:
df.sort_values(by=["Criteria"])

,Criteria,Weight
6,a,24.183007
0,b,2.614379
2,c,6.209150
1,d,2.614379
3,e,13.398693
4,f,13.398693
7,g,24.183007
5,h,13.398693
Total,NaN,100.000000


In [21]:
df.sort_values(by=["Weight"])

,Criteria,Weight
0,b,2.614379
1,d,2.614379
2,c,6.209150
3,e,13.398693
4,f,13.398693
5,h,13.398693
6,a,24.183007
7,g,24.183007
Total,NaN,100.000000


# Examples of use

## Example 1

`subsets_1.csv` data from:

> Papathanasiou, J., Ploskas, N. (2018). Multiple criteria decision aid. Methods, Examples and Python Implementations, 136.Appendix: Revised Simos https://doi.org/10.1007/978-3-319-91648-4
>
> Table A.1 Ranking of the criteria using cards, page 167
> Table A.2 Criteria weights, page 168

In [22]:
set_cards = "./data/subsets_1.csv"

print("Set of cards (Table A.1, page 167)")
pd.read_csv(set_cards).fillna('')

Set of cards (Table A.1, page 167)


FileNotFoundError: [Errno 2] No such file or directory: './data/subsets_1.csv'

In [ ]:
# The last subset is z = 6.5 more important than the 1st one
z = 6.5

df = Simos_revised.criteria_weights_Simos(set_cards, z)

print(f"\nResults for {set_cards} with z = {z} (Table A.2, page 168):\n")
print("Sorted criteria ")
df.sort_values(by=["Criteria"])

In [ ]:
print("Sorted weights")
df.sort_values(by=["Weight"])

## Example 2

`subsets_2.csv` data from:

> Figueira, J., Roy, B. (2002). Determining the weights of criteria in the ELECTRE type methods with a revised Simos' procedure. European journal of operational research, 139(2), 317-326. https://doi.org/10.1016/S0377-2217(01)00370-8
>
> Table 1 Presentation of the information given by the set of cards, page 319
> 
> Table 5, Determining the normalized weights of each criterion for w = 1 and z = 6.5, column *Normalized weights $k^*_i$* (note that in this table, the results are in the order of the weights, not of the criteria), page 324)

In [ ]:
import Simos_revised

set_cards = "./data/subsets_2.csv"

print("Set of cards (Table 1, page 319)")
pd.read_csv(set_cards).fillna('')

In [ ]:
# The last subset is z = 6.5 more important than the 1st one
z = 6.5

df = Simos_revised.criteria_weights_Simos(set_cards, z)

print(f"\nResults for {set_cards} with z = {z} (Table 5, page 324):\n")
print("Sorted weights")
df.sort_values(by=["Weight"])

# Bibliography

Papathanasiou, J., Ploskas, N. (2018). Multiple criteria decision aid. Methods, Examples and Python Implementations, 136.Appendix: Revised Simos https://doi.org/10.1007/978-3-319-91648-4

Figueira, J., Roy, B. (2002). Determining the weights of criteria in the ELECTRE type methods with a revised Simos' procedure. European journal of operational research, 139(2), 317-326. https://doi.org/10.1016/S0377-2217(01)00370-8